# 지오코딩_공급처(전기차 충전소) 지도 시각화 작업

### 데이터를 지도를 통해 직관적으로 확인할 수 있는 시각화를 원했다. 지도 시각화를 위한 각종 라이브러리를 사용하기 위해선 위/경도 추출이 꼭 필요했다.

### kakao map의 RESTful API를 활용하여, 주소값을 이용해 위/경도를 추출할 수 있다는걸 알게 됨.

In [5]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 1. CSV 파일 불러오기 (예: 제주도 충전소 데이터)
df = pd.read_csv("서울_충전소.csv", encoding="utf-8-sig")  # 파일명만 바꿔서 재사용 가능
# 2. Kakao REST API 키 입력
api_key = "Your api key"  # 여기에 발급받은 REST API 키를 입력

# 3. 주소 → 위도/경도 요청 함수 정의
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']  # 위도(y), 경도(x)
        return None, None
    except Exception as e:
        return None, None

# 4. 주소 리스트 추출
addresses = df["주소"].tolist()

# 5. 병렬 처리 시작
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

# 6. 결과 DataFrame에 추가
df["위도"] = lat_list
df["경도"] = lng_list

# 7. 결과 저장
df.to_csv("서울_충전소_좌표추가.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 서울_충전소_좌표추가.csv")


100%|████████████████████████████████████████████████████████████████████████████| 46175/46175 [11:49<00:00, 65.12it/s]


✔️ 저장 완료: 서울_충전소_좌표추가.csv


In [7]:
print(lat_list[:10])  # 앞쪽 위도값 10개만 출력
print(lng_list[:10])  # 앞쪽 경도값 10개만 출력

[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]


In [13]:
# 디버깅 작업
import requests

def debug_lat_lng(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)
    print("응답코드:", response.status_code)
    print("응답내용:", response.json())

# 예시 주소로 테스트
api_key = "your api key"
test_address = "서울특별시 강남구 역삼동 822-1"
debug_lat_lng(test_address, api_key)


응답코드: 200
응답내용: {'documents': [{'address': {'address_name': '서울 강남구 역삼동 822-1', 'b_code': '1168010100', 'h_code': '1168064000', 'main_address_no': '822', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_h_name': '역삼1동', 'region_3depth_name': '역삼동', 'sub_address_no': '1', 'x': '127.028512212479', 'y': '37.4986639396111'}, 'address_name': '서울 강남구 역삼동 822-1', 'address_type': 'REGION_ADDR', 'road_address': {'address_name': '서울 강남구 테헤란로 105', 'building_name': '', 'main_building_no': '105', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '역삼동', 'road_name': '테헤란로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.028529843894', 'y': '37.4986369952659', 'zone_no': '06134'}, 'x': '127.028512212479', 'y': '37.4986639396111'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [15]:
# API 제한으로 인한 오류로 확인하고 30,000개로 제한해서 호출 후 작업
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# ✅ 설정: 처리할 구간 지정
start_idx = 0        # 시작 인덱스
end_idx = 30000      # 종료 인덱스 (30,000건 이하)

# ✅ 1. 전체 CSV에서 부분만 불러오기
df_all = pd.read_csv("서울_충전소.csv", encoding="utf-8-sig")
df = df_all.iloc[start_idx:end_idx].copy() # copy() 쓰는것 중요. copy()안쓰면 df는 df_all의 view가 되어 df 변경 시, df_all도 변경됨

# ✅ 2. API 키
api_key = "69e904e85329cddd6795bf2cfe6c576a"

# ✅ 3. 지오코딩 함수 정의
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']
        return None, None
    except Exception:
        return None, None

# ✅ 4. 주소 리스트 추출 및 병렬 처리
addresses = df["주소"].tolist()
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses}
    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

# ✅ 5. 결과 추가
df["위도"] = lat_list
df["경도"] = lng_list

# ✅ 6. 부분 저장
df.to_csv(f"서울_충전소_좌표추가_{start_idx}_{end_idx - 1}.csv", index=False, encoding="utf-8-sig")
print(f"✔️ 저장 완료: 서울_충전소_좌표추가_{start_idx}_{end_idx - 1}.csv")


100%|████████████████████████████████████████████████████████████████████████████| 30000/30000 [08:14<00:00, 60.73it/s]


✔️ 저장 완료: 서울_충전소_좌표추가_0_29999.csv


In [1]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 1. CSV 파일 불러오기
df = pd.read_csv("서울_충전소.csv", encoding="utf-8-sig")

# 2. Kakao REST API 키
api_key = "69e904e85329cddd6795bf2cfe6c576a"  # 예: "69e904e85329cddd6795bf2cfe6c576a"

# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']  # 위도(y), 경도(x)
        return None, None
    except:
        return None, None

# 4. 전체 주소 중, 30,000번째 이후만 추출
addresses = df["주소"].tolist()[30000:]  # 인덱스 30000부터 끝까지

# 5. 병렬 처리
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

# 6. 부분 결과 DataFrame 생성
df_partial = df.iloc[30000:].copy()
df_partial["위도"] = lat_list
df_partial["경도"] = lng_list

# 7. 저장
df_partial.to_csv("서울_충전소_좌표추가_2차.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 서울_충전소_좌표추가_2차.csv")


100%|███████████████████████████████████████████████████████████████████████████| 16175/16175 [02:13<00:00, 121.38it/s]

✔️ 저장 완료: 서울_충전소_좌표추가_2차.csv


In [1]:
# 벌도의 작업 이후 병합
import pandas as pd

# 1. 두 CSV 파일 불러오기
df1 = pd.read_csv("서울_충전소_좌표추가_1차.csv", encoding="utf-8-sig")
df2 = pd.read_csv("서울_충전소_좌표추가_2차.csv", encoding="utf-8-sig")

# 2. 병합
df_merged = pd.concat([df1, df2], ignore_index=True) # ignore_index를 True 하면 두 인덱스를  합치지 않고, 0, 1, 2...처럼 새로운 연속적인 인덱스를 부여함
df_merged = df_merged.drop_duplicates() # drop_duplicates()는 행의 내용이 완전이 동일한 중복된 내용을 제거하고 유일한 내용만 남기는 메서드

# 3. 저장
df_merged.to_csv("서울_충전소_좌표추가_전체.csv", index=False, encoding="utf-8-sig")
print("✔️ 최종 저장 완료: 서울_충전소_좌표추가_전체.csv")


✔️ 최종 저장 완료: 서울_충전소_좌표추가_전체.csv


In [3]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 1. CSV 파일 불러오기
df = pd.read_csv("제주도_충전소.csv", encoding="utf-8-sig")

# 2. Kakao REST API 키
api_key = "69e904e85329cddd6795bf2cfe6c576a"  # 예: "69e904e85329cddd6795bf2cfe6c576a"

# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']  # 위도(y), 경도(x)
        return None, None
    except:
        return None, None

# 4. 전체 주소 중, 30,000번째 이후만 추출
addresses = df["주소"].tolist()  # 인덱스 

# 5. 병렬 처리
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

# 6. 부분 결과 DataFrame 생성
df["위도"] = lat_list
df["경도"] = lng_list

# 7. 저장
df.to_csv("제주도_충전소_좌표추가_전체.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 제주도_충전소_좌표추가_전체.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 6473/6473 [00:40<00:00, 160.01it/s]

✔️ 저장 완료: 제주도_충전소_좌표추가_전체.csv


In [5]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 1. CSV 파일 불러오기
df = pd.read_csv("인천광역시_충전소.csv", encoding="utf-8-sig")

# 2. Kakao REST API 키
api_key = "69e904e85329cddd6795bf2cfe6c576a"  # 예: "69e904e85329cddd6795bf2cfe6c576a"

# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']  # 위도(y), 경도(x)
        return None, None
    except:
        return None, None

# 4. 전체 주소 중, 30,000번째 이후만 추출
addresses = df["주소"].tolist()  # 인덱스 

# 5. 병렬 처리
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

# 6. 부분 결과 DataFrame 생성
df["위도"] = lat_list
df["경도"] = lng_list

# 7. 저장
df.to_csv("인천광역시_충전소_좌표추가_전체.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 인천광역시_충전소_좌표추가_전체.csv")

100%|███████████████████████████████████████████████████████████████████████████| 19946/19946 [02:04<00:00, 160.75it/s]


✔️ 저장 완료: 인천광역시_충전소_좌표추가_전체.csv


In [ ]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 1. CSV 파일 불러오기
df = pd.read_csv("경기도_충전소.csv", encoding="utf-8-sig")

# 2. Kakao REST API 키
api_key = "Your api key"  

# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']  # 위도(y), 경도(x)
        return None, None
    except:
        return None, None

# 4. 전체 주소 중, 30,000번째 이후만 추출
addresses = df["주소"].tolist()  # 인덱스 

# 5. 병렬 처리
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

# 6. 부분 결과 DataFrame 생성
df["위도"] = lat_list
df["경도"] = lng_list

# 7. 저장
df.to_csv("경기도_충전소_좌표추가_전체.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 경기도_충전소_좌표추가_전체.csv")

In [7]:
pip install folium

In [11]:
import folium # folium : python에서 인터렉티브한 웹 지도를 만들 수 있게 해주는 라이브러리

# 서울 중심부를 기준으로 초기 지도 설정 (위도, 경도는 너가 원하는 위치로)
map = folium.Map(location=[37.5665, 126.9780], zoom_start=11)  # 서울 시청 기준
map # jupyternote 환경에서 지도 열기

In [ ]:
import pandas as pd
import folium

df = pd.read_csv("서울_충전소_좌표추가_전체.csv")
df = df.dropna(subset=["위도", "경도"]).reset_index(drop=True)  # dropna 이후 reset_index를 통해 인덱스 재설정 해줘야 함!

map = folium.Map(location=[df["위도"].mean(), df["경도"].mean()], zoom_start=11)

for i in range(len(df)):
    lat = df.loc[i, "위도"]
    lng = df.loc[i, "경도"]
    name = df.loc[i, "충전소명"] if "충전소명" in df.columns else f"충전소 {i+1}"

    folium.CircleMarker(
        location=[lat, lng],
        radius=0.5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=name
    ).add_to(map)

map.save("서울_충전소_지도.html")  # 결과를 HTML로 저장해 웹에서 볼 수 있게 함

## 결측값(NaN) 제거

In [13]:

regions = ["서울", "경기도", "인천광역시", "제주도"]

for region in regions:
    file_path = f"{region}_충전소_좌표추가_전체.csv"
    df = pd.read_csv(file_path)
    df_cleaned = df.dropna(subset=["위도", "경도"]).reset_index(drop=True)
    df_cleaned.to_csv(f"{region}_충전소_좌표추가_전체_cleaned.csv", index=False, encoding='utf-8-sig')



## 지도에 전체 데이터셋(서울/경기/인천/제주) 표현

In [17]:
import pandas as pd
import folium

df = pd.read_csv("경기도_충전소_좌표추가_전체_cleaned.csv")

map = folium.Map(location=[df["위도"].mean(), df["경도"].mean()], zoom_start=11)

for i in range(len(df)):
    lat = df.loc[i, "위도"]
    lng = df.loc[i, "경도"]
    name = df.loc[i, "충전소명"] if "충전소명" in df.columns else f"충전소 {i+1}"

    folium.CircleMarker(
        location=[lat, lng],
        radius=0.3,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=name
    ).add_to(map)

map.save("경기도_충전소_지도.html")  # 결과를 HTML로 저장해 웹에서 볼 수 있게 함

In [21]:
import pandas as pd
import folium

df = pd.read_csv("인천광역시_충전소_좌표추가_전체_cleaned.csv")

# folium.Map(): folium 라이브러리 새로운 지도 객체 생성 함수
map = folium.Map(location=[df["위도"].mean(), df["경도"].mean()], zoom_start=11) # 초기 html 파일 open시, 줌 설정

for i in range(len(df)):
    lat = df.loc[i, "위도"]
    lng = df.loc[i, "경도"]
    name = df.loc[i, "충전소명"] if "충전소명" in df.columns else f"충전소 {i+1}"

    folium.CircleMarker(
        location=[lat, lng],
        radius=0.3,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        popup=name
    ).add_to(map)

map.save("인천광역시_충전소_지도.html")  # 결과를 HTML로 저장해 웹에서 볼 수 있게 함

## 전체 충전소 지도 데이터셋 합치기

In [24]:
import pandas as pd

# 병합할 파일 경로 목록
file_paths = [
    "서울_충전소_좌표추가_전체_cleaned.csv",
    "경기도_충전소_좌표추가_전체_cleaned.csv",
    "인천광역시_충전소_좌표추가_전체_cleaned.csv",
    "제주도_충전소_좌표추가_전체_cleaned.csv"
]

# 각 파일을 읽어서 리스트에 저장
dfs = [pd.read_csv(path, encoding="utf-8-sig") for path in file_paths]

# 데이터프레임 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 중복 제거 및 인덱스 초기화
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

# 병합 결과 저장 (선택사항)
merged_df.to_csv("전국_충전소_좌표통합.csv", index=False, encoding="utf-8-sig")

print("✔️ 병합 완료: 전국_충전소_좌표통합.csv")


✔️ 병합 완료: 전국_충전소_좌표통합.csv


## 전체 데이터셋 지도 표시

In [27]:
import pandas as pd
import folium

df = pd.read_csv("전국_충전소_좌표통합.csv")

map = folium.Map(location=[df["위도"].mean(), df["경도"].mean()], zoom_start=11)

for i in range(len(df)):
    lat = df.loc[i, "위도"]
    lng = df.loc[i, "경도"]
    name = df.loc[i, "충전소명"] if "충전소명" in df.columns else f"충전소 {i+1}"

    folium.CircleMarker(
        location=[lat, lng],
        radius=0.3,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.2,
        popup=name
    ).add_to(map)

map.save("전국_충전소_지도.html")  # 결과를 HTML로 저장해 웹에서 볼 수 있게 함